In [1]:
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn import metrics

import numpy as np
import pandas as pd
import re

Innetől a beolvasáshoz segédlet

In [33]:
df_losses = pd.read_csv('model_losses.csv')
# One column
# p30_u10 - configuration key: Percentile 30, Uniformity 10
# nature - model type: nature, urban or all
# all - dataset type: nature, urban or all
# 0.6424242854118347 - loss value for picture 0
# 15 more float values for 15 more pictures
df_actual = pd.read_csv('image_actual.csv')

In [34]:
for col in df_losses.columns:
    # get first row value for this specific column
    col_name = re.sub('\.\d$', '', col)
    first_row = df_losses.iloc[0][col]
    second_row = df_losses.iloc[1][col]
    new_column_name = f"{col_name}_{first_row}_{second_row}"
    # rename the column with the existing column header plus the first row of that column's data
    df_losses.rename(columns={col: new_column_name}, inplace=True)

# delete first row of data, now that it's merged into the header
df_losses.drop(index=[0,1], inplace=True)
df_losses.drop(columns=df_losses.columns[0], axis=1, inplace=True)
df_losses.reset_index(drop=True, inplace=True)
df_losses=df_losses.astype(float)
for col in df_actual.columns:
    # get first row value for this specific column
    col_name = re.sub('\.\d$', '', col)
    first_row = df_actual.iloc[0][col]
    new_column_name = f"{col_name}_{first_row}"
    # rename the column with the existing column header plus the first row of that column's data
    df_actual.rename(columns={col: new_column_name}, inplace=True)

# delete first row of data, now that it's merged into the header
df_actual.drop(index=[0], inplace=True)
df_actual.drop(columns=df_actual.columns[0], axis=1, inplace=True)
df_actual.reset_index(drop=True,inplace=True)
df_actual=df_losses.astype(float)

In [35]:
df_losses_true = pd.DataFrame(df_losses, index=df_losses.index, columns=df_losses.columns)
df_actual_true = pd.DataFrame(df_actual, index=df_actual.index, columns=df_actual.columns)

In [36]:
df_losses_true

,p30_u10_nature_all,p30_u10_nature_nature,p30_u10_nature_urban,p30_u10_urban_all,p30_u10_urban_nature,p30_u10_urban_urban,p30_u10_all_all,p30_u10_all_nature,p30_u10_all_urban,p50_u10_nature_all,...,p50_u40_all_urban,p70_u40_nature_all,p70_u40_nature_nature,p70_u40_nature_urban,p70_u40_urban_all,p70_u40_urban_nature,p70_u40_urban_urban,p70_u40_all_all,p70_u40_all_nature,p70_u40_all_urban
0,0.642424,0.547332,0.637886,0.642424,0.547332,0.637886,0.614295,0.540134,0.543107,0.642424,...,0.476015,0.707408,0.707510,0.707499,0.707408,0.707510,0.707499,0.614295,0.580301,0.543107
1,0.635478,0.636928,0.372586,0.635478,0.636928,0.372586,0.765808,0.605552,0.310331,0.635478,...,0.602613,0.707542,0.707570,0.707532,0.707542,0.707570,0.707532,0.765808,0.599912,0.564291
2,0.545241,0.553180,0.372142,0.545241,0.553180,0.372142,0.517023,0.520848,0.311279,0.545241,...,0.531933,0.707475,0.707552,0.707519,0.707475,0.707552,0.707519,0.517023,0.600874,0.506092
3,0.656434,0.636527,0.627115,0.656434,0.636527,0.627115,0.610423,0.641761,0.566174,0.656434,...,0.618435,0.707565,0.707552,0.707562,0.707565,0.707552,0.707562,0.610423,0.603428,0.553069
4,0.379647,0.690713,0.636912,0.379647,0.690713,0.636912,0.320557,0.748589,0.561785,0.379647,...,0.596629,0.707378,0.707557,0.707452,0.707378,0.707557,0.707452,0.320557,0.610456,0.259857
5,0.637923,0.646411,0.652575,0.637923,0.646411,0.652575,0.637429,0.600853,0.582651,0.637923,...,0.643120,0.707488,0.707573,0.707487,0.707488,0.707573,0.707487,0.637429,0.718583,0.475934
6,0.663855,0.631923,0.647813,0.663855,0.631923,0.647813,0.636445,0.649667,0.641723,0.663855,...,0.587869,0.707573,0.707555,0.707556,0.707573,0.707555,0.707556,0.636445,0.583194,0.597890
7,0.622270,0.648376,0.628178,0.622270,0.648376,0.628178,0.620611,0.616718,0.565424,0.622270,...,0.613635,0.707485,0.707487,0.707375,0.707485,0.707487,0.707375,0.620611,0.619347,0.286996
8,0.625908,0.643473,0.652314,0.625908,0.643473,0.652314,0.582181,0.663073,0.632765,0.625908,...,0.621586,0.707524,0.707511,0.707378,0.707524,0.707511,0.707378,0.582181,0.578476,0.323328
9,0.554281,0.643329,0.290546,0.554281,0.643329,0.290546,0.521410,0.584309,0.259650,0.554281,...,0.602041,0.707480,0.707556,0.707557,0.707480,0.707556,0.707557,0.521410,0.611876,0.308762


In [39]:
for col in df_losses_true.columns:
  df_losses_true.loc[df_losses_true[col] > 0.5, col] = 1
  df_losses_true.loc[df_losses_true[col] <= 0.5, col] = 0
for col in df_actual_true.columns:
  df_actual_true.loc[df_actual_true[col] > 0.5, col] = 1
  df_actual_true.loc[df_actual_true[col] <= 0.5, col] = 0

In [58]:
#A tesztelni kívánt oszlopot beolvassuk mindkét adaktészletből
loses = np.array(df_losses_true['p50_u10_nature_all'])
act = np.array(df_actual_true['p50_u10_nature_all'])

Beolvasás és átalakítások vége. Innentől a már korébban meglévő confusion mátrix és score-ok számítása következik.

In [54]:
tn, fp, fn, tp  = metrics.confusion_matrix(act,loses).ravel() #roc_curve(test, pred, drop_intermediate=False)
print(tn, fp, fn, tp)

5 0 0 11


In [55]:
#Intersection over Union
IoU = tp / (tp+fn+fp)
IoU

1.0

In [56]:
#Sørensen–Dice
DSC = (2*tp) / (2*tp+fn+fp)
DSC

1.0